In [42]:
import os
import pandas as pd
import numpy as np
import requests


In [43]:
CLIMDIV_PRCP_URL = "https://www.ncei.noaa.gov/pub/data/cirs/climdiv/climdiv-pcpnst-v1.0.0-20250905"
CLIMDIV_TEMP_URL = "https://www.ncei.noaa.gov/pub/data/cirs/climdiv/climdiv-tmpcst-v1.0.0-20250905"

USDA_BASE_URL = "https://quickstats.nass.usda.gov/api/api_GET/"

DEFAULT_STATES = ["IA","IL","IN","OH","MO","MN","NE"]
USDA_API_KEY = "D1ABF2AD-362D-346E-A641-93A2FA6ED6D8"

MAPPING = {
        1:"AL",  2:"AZ",  3:"AR",  4:"CA",  5:"CO",  6:"CT",  7:"DE",  8:"FL",  9:"GA",
    10:"ID", 11:"IL", 12:"IN", 13:"IA", 14:"KS", 15:"KY", 16:"LA", 17:"ME", 18:"MD",
    19:"MA", 20:"MI", 21:"MN", 22:"MS", 23:"MO", 24:"MT", 25:"NE", 26:"NV", 27:"NH",
    28:"NJ", 29:"NM", 30:"NY", 31:"NC", 32:"ND", 33:"OH", 34:"OK", 35:"OR", 36:"PA",
    37:"RI", 38:"SC", 39:"SD", 40:"TN", 41:"TX", 42:"UT", 43:"VT", 44:"VA", 45:"WA",
    46:"WV", 47:"WI", 48:"WY", 49:"HI", 50:"AK", 110:"US"
    }

In [44]:
def ensure_dirs():
    for p in ["data/raw","data/interim","data/processed"]:
        os.makedirs(p, exist_ok=True)

ensure_dirs()

def month_name(m):
    return ["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"][m]


def ensure_dirs():
    for p in ["data/raw","data/interim","data/processed"]:
        os.makedirs(p, exist_ok=True)

ensure_dirs()

def month_name(m):
    return ["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"][m]


def add_shortfall(df):
    stats = df.groupby("state")["prec_jun"].agg(mean_jun="mean", p10=lambda x: np.percentile(x, 10)).reset_index()
    df = df.merge(stats, on="state", how="left")
    df["jun_shortfall"] = np.where(df["prec_jun"] <= df["p10"], df["prec_jun"] - df["mean_jun"], 0.0)
    df["jun_shortfall"] = 0
    df.loc[df["year"].isin([1988, 2012]), "jun_shortfall"] = (
        df["prec_jun"] - df["prec_jun"].mean()
    )
    print("ok add shortfall")
    return df.drop(columns=["mean_jun","p10"])

In [45]:
def load_weather(url):
    resp = requests.get(url)
    resp.raise_for_status()
    rows = []

    for line in resp.text.splitlines():
        line = line.strip()
        if not line:
            print("pas de ligne")
            continue

        state_code = int(line[0:3])
        division = int(line[3:4])
        year = int(line[6:10])

        if state_code > 50 and state_code != 110:
            continue
        
        parts = line[8:].split()
        if len(parts) < 12:
            print("moins de 12 cols")
            continue  
        values = [float(x) for x in parts[:12]]
        
        rows.append([state_code, division, year] + values)

    cols = ["state_code", "division", "year"] + [f"m{m:02d}" for m in range(1, 13)]
    df = pd.DataFrame(rows, columns=cols)

    

    return df

def climdiv_statewide_month(df):
    grp = df.groupby(["state_code","year"]).mean(numeric_only=True).reset_index()
    long = grp.melt(id_vars=["state_code","year"], value_vars=[f"m{m:02d}" for m in range(1,13)],
                    var_name="month", value_name="value")
    long["month"] = long["month"].str[1:].astype(int)
    print("ok climdiv srtatewide mothn")
    long["state"] = long["state_code"].map(MAPPING)
    return long



def build_weather_table(start, end, states):
    tdf = load_weather(CLIMDIV_TEMP_URL)
    tdf.to_csv("data/raw/tdf.csv", index=False)
    pdf = load_weather(CLIMDIV_PRCP_URL)
    pdf.to_csv("data/raw/pdf.csv", index=False)

    t_long = climdiv_statewide_month(tdf)
    p_long = climdiv_statewide_month(pdf)

    t_long = t_long.dropna(subset=["state"])
    p_long = p_long.dropna(subset=["state"])
    
    t_long = t_long[(t_long["year"].between(start, end)) & (t_long["state"].isin(states))]
    p_long = p_long[(p_long["year"].between(start, end)) & (p_long["state"].isin(states))]

    
    T = t_long.pivot_table(index=["state","year"], columns="month", values="value").reset_index()
    P = p_long.pivot_table(index=["state","year"], columns="month", values="value").reset_index()
    
    T.columns = ["state","year"] + [f"t_{month_name(m).lower()}" for m in range(1,13)]
    P.columns = ["state","year"] + [f"p_{month_name(m).lower()}" for m in range(1,13)]

    T.to_csv("data/interim/t_long.csv", index=False)
    P.to_csv("data/interim/p_long.csv", index=False)
    
    W = pd.merge(T, P, on=["state","year"], how="inner")
    W['temp_JA'] = W[['t_jul','t_aug']].mean(axis=1)
    W['prec_JA'] = W[['p_jul','p_aug']].mean(axis=1)
    W["prec_jun"] = W["p_jun"]
    W['prec_JA_sq'] = 0
    
    print("ok build weather")
    return W[["state","year","temp_JA","prec_JA","prec_JA_sq","prec_jun"]]



In [46]:
def usda_quickstats(params):
    key = USDA_API_KEY
    if not key:
        raise RuntimeError("USDA API key not found")
    params = dict(params)
    params["key"] = key
    r = requests.get(USDA_BASE_URL, params=params, timeout=60)
    r.raise_for_status()
    return pd.DataFrame(r.json()["data"])

def get_soy_state_metric(start, end, states, metric, unit, out_col, agg="sum"):
    params = {
        "commodity_desc": "SOYBEANS",
        "statisticcat_desc": metric,
        "unit_desc": unit,
        "agg_level_desc": "STATE",
        "sector_desc": "CROPS",
        "group_desc": "FIELD CROPS",
        "source_desc": "SURVEY",
        "year__GE": start,
        "year__LE": end
    }
    df = usda_quickstats(params)
    df = df[df["state_alpha"].isin(states)].copy()
    df["Value"] = pd.to_numeric(df["Value"].str.replace(",", ""), errors="coerce")

    if agg == "mean":
        out = df.groupby(["state_alpha", "year"], as_index=False)["Value"].mean()
    else:
        out = df.groupby(["state_alpha", "year"], as_index=False)["Value"].sum()

    out.rename(columns={"state_alpha": "state", "Value": out_col}, inplace=True)
    return out


In [47]:
def build_state_features(start, end, states):
    W = build_weather_table(start, end, states)
    Y = get_soy_state_metric(start, end, states, "YIELD", "BU / ACRE", "yield_bu_acre", "mean")
    Y.to_csv("data/raw/yield.csv", index=False)
    A = get_soy_state_metric(start, end, states, "AREA HARVESTED", "ACRES", "acres_harvested", "sum")
    A.to_csv("data/raw/harvest.csv", index=False)
    df = W.merge(Y, on=["state","year"]).merge(A, on=["state","year"])
    df = add_shortfall(df)
    df["trend"] = df["year"] - 1987
    df["dummy_2003"] = (df["year"]==2003).astype(int)
    cols = ["state","year","yield_bu_acre","trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq","dummy_2003","acres_harvested"]

    print("ok build state feature")

    print(W.head())
    return df[cols]


data = build_state_features(1988, 2024, ["IA","IL","IN","OH","MO"])
data.to_csv("data/processed/waob_features_states.csv", index=False)

ok climdiv srtatewide mothn
ok climdiv srtatewide mothn
ok build weather
ok add shortfall
ok build state feature
  state  year  temp_JA  prec_JA  prec_JA_sq  prec_jun
0    IA  1988    74.65     1.99           0      1.77
1    IA  1989    71.35     3.35           0      2.57
2    IA  1990    71.30     7.16           0      5.73
3    IA  1991    73.30     3.23           0      5.19
4    IA  1992    67.85     5.22           0      1.75


C:\Users\agerard\AppData\Local\Temp\ipykernel_24500\3022769094.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-2.94697297 -1.20697297 -3.01697297 -2.24697297 -3.17697297 -1.96697297
 -2.41697297 -2.55697297 -2.99697297 -1.03697297]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df["year"].isin([1988, 2012]), "jun_shortfall"] = (


In [49]:
def aggregate_national(df_state, method="weighted"):
    rows = []
    for year, grp in df_state.groupby("year"):
        w = grp["acres_harvested"] if method=="weighted" else pd.Series(1.0, index=grp.index)
        def wavg(s): return np.average(s, weights=w)
        p_ja = np.average(grp['prec_JA'], weights=w)
        rows.append({
            "year": year,
            "yield_bu_acre": wavg(grp["yield_bu_acre"]),
            "trend": wavg(grp["trend"]),
            "jun_shortfall": wavg(grp["jun_shortfall"]),
            "temp_JA": wavg(grp["temp_JA"]),
            'prec_JA': p_ja,
            'prec_JA_sq': p_ja ** 2, 
            "dummy_2003": int(year==2003),
            "acres_harvested": grp["acres_harvested"].sum()
        })
    return pd.DataFrame(rows).sort_values("year")

df_nat = aggregate_national(data)
df_nat.to_csv("data/processed/waob_features_national.csv", index=False)
print(df_nat.tail())

    year  yield_bu_acre  trend  jun_shortfall    temp_JA   prec_JA  \
32  2020      56.893384   33.0            0.0  74.941900  4.163177   
33  2021      59.272252   34.0            0.0  73.876901  4.796157   
34  2022      58.018797   35.0            0.0  74.182275  3.786644   
35  2023      57.447629   36.0            0.0  72.620747  3.408518   
36  2024      59.745887   37.0            0.0  73.530513  4.594102   

    prec_JA_sq  dummy_2003  acres_harvested  
32   17.332040           0        214940000  
33   23.003118           0        220700000  
34   14.338670           0        226630000  
35   11.617993           0        214660000  
36   21.105774           0        223460000  
